In [7]:
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
import requests
import argparse
import typing
import json
from urllib.parse import quote_plus
from src.utils import df_na_vals, np_min_chunk, time_utc_now
from src.fetch import apiurl
import os

In [5]:
time_utc_now()

Timestamp('2023-09-17 05:47:23')

In [32]:
from src import const
import logging

query = {"external": {"eq": False}, "disable_bundling": {"eq": True}, "type": "on-demand"}
url = apiurl(const.VAST_API_URL, "/bundles", {"q": query})

url = const.VAST_EXPORTER_URL + '/offers'

r = requests.get(url, timeout=20)
r.raise_for_status()


ts = int(pd.to_datetime(r_dict.get('timestamp', time_utc_now())).timestamp())
raw = pd.DataFrame(r.json()["offers"])
raw['timestamp'] = ts
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6028 entries, 0 to 6027
Data columns (total 61 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bundle_id             6028 non-null   int64  
 1   bw_nvlink             6028 non-null   float64
 2   client_run_time       6028 non-null   float64
 3   compute_cap           6028 non-null   int64  
 4   cpu_cores             6028 non-null   int64  
 5   cpu_cores_effective   6028 non-null   float64
 6   cpu_name              5973 non-null   object 
 7   cpu_ram               6028 non-null   int64  
 8   credit_balance        0 non-null      object 
 9   credit_discount       0 non-null      object 
 10  credit_discount_max   5186 non-null   float64
 11  cuda_max_good         6028 non-null   float64
 12  direct_port_count     6028 non-null   int64  
 13  discount_rate         5186 non-null   float64
 14  discounted_dph_total  6028 non-null   float64
 15  discounted_hourly    

In [24]:
r_dict.keys()

dict_keys(['offers'])

In [ ]:
r.text

In [20]:
# !python ../vast search offers "rentable = any verified = any" --raw
# !python ../vast search offers "machine_id = 9901 rentable = any verified = any" --raw -i
!python ../vast search offers "machine_id = 6312 rentable = any verified = any" --raw

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%226312%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '6312'}, 'order': [['score', 'desc']], 'type': 'on-demand'}
[
 {
  "bundle_id": 321281227,
  "bundled_results": 4,
  "bw_nvlink": 56.24800109863281,
  "client_run_time": 1.1,
  "compute_cap": 860,
  "cpu_cores": 128,
  "cpu_cores_effective": 64.0,
  "cpu_name": null,
  "cpu_ram": 257585,
  "credit_balance": null,
  "credit_discount": null,
  "credit_discount_max": 0.4,
  "cuda_max_good": 11.8,
  "direct_port_count": 1000,
  "discount_rate": 0.0,
  "discounted_dph_total": 0.8,
  "discounted_hourly": 0.0,
  "disk_bw": 933.8698507247343,
  "disk_name": "AMI Virtual",
  "disk_space": 345.15,
  "dlperf": 61.523666,
  "dlperf_per_dphtotal": 76.90

In [101]:
!python vast search offers "machine_id = 5919 rentable = any verified = any" -d
# !python vast search offers "machine_id = 3919" -d

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%225919%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '5919'}, 'order': [['score', 'desc']], 'type': 'on-demand'}
ID       CUDA   N  Model  PCIE  vCPUs    RAM  Disk  $/hr    DLP    DLP/$  NV Driver  Net_up  Net_down  R     Max_Days  mach_id  status    ports  country       
5402875  11.8  4x  A40    24.2  128.0  257.6  672   1.9600  122.5  62.48  520.61.05  503.5   667.0     99.9  -         5919     verified  2000   New_Jersey,_US
5516880  11.8  2x  A40    24.3  64.0   257.6  336   0.9800  61.2   62.45  520.61.05  503.5   667.0     99.9  -         5919     verified  1000   New_Jersey,_US
5516875  11.8  1x  A40    24.2  32.0   257.6  168   0.4900  30.6   62.44  520.61.05  503.5   667.0     99.9 

In [97]:
# !python vast search offers "rentable = any verified = any" --raw -d
# !python vast search offers "machine_id = 11833 rentable = any verified = any" --raw -d
!python vast search offers "machine_id = 5919 rentable = any verified = any" --raw -d

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%225919%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '5919'}, 'order': [['score', 'desc']], 'type': 'on-demand'}
[
 {
  "bundle_id": 313989942,
  "bundled_results": 8,
  "bw_nvlink": 56.24800109863281,
  "compute_cap": 860,
  "cpu_cores": 128,
  "cpu_cores_effective": 32.0,
  "cpu_name": null,
  "cpu_ram": 257585,
  "credit_balance": null,
  "credit_discount": null,
  "credit_discount_max": null,
  "cuda_max_good": 11.8,
  "direct_port_count": 500,
  "discount_rate": null,
  "discounted_dph_total": 0.49,
  "discounted_hourly": 0,
  "disk_bw": 1496.2075513316365,
  "disk_name": "AMI Virtual",
  "disk_space": 168.075,
  "dlperf": 30.597611,
  "dlperf_per_dphtotal": 62.44410408163265,
  "dph_bas

In [4]:
# query = {"external": {"eq": False},  "disable_bundling" : {"eq": True}, "type": "bid"}
query = {"external": {"eq": False},  "disable_bundling" : {"eq": True}, "type": "on-demand"}
# query = {"external": {"eq": False}, "type": "on-demand"}

# query = {'external': {'eq': False}, 'machine_id': {'eq': '3690'}, 'type': 'on-demand'}
# query = {'external': {'eq': False}, 'machine_id': {'eq': '6312'}, "disable_bundling" : {"eq": True}, 'type': 'on-demand'}

url = apiurl("/bundles", {"q": query})
url

'https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22disable_bundling%22%3A+%7B%22eq%22%3A+true%7D%2C+%22type%22%3A+%22on-demand%22%7D'

ImportError: cannot import name 'split_raw' from partially initialized module 'src.preprocess' (most likely due to a circular import) (C:\Users\ergot\DataspellProjects\vast-stats\src\preprocess.py)

In [7]:
r.json().keys()

dict_keys(['offers'])

In [25]:
df  = pd.DataFrame(rows)
df[['machine_id', 'id', 'num_gpus', 'rentable']].sort_values('id')

,machine_id,id,num_gpus,rentable
13,6312,5381479,1,False
12,6312,5381480,1,False
11,6312,5381481,1,False
1,6312,5381482,4,False
5,6312,5381483,2,False
10,6312,5381484,1,False
4,6312,5381485,2,False
9,6312,5516883,1,False
8,6312,5516884,1,False
7,6312,5516885,1,False


In [26]:
df  = pd.DataFrame(rows)
df[['machine_id', 'id', 'num_gpus', 'rentable']]
# df.set_index(['machine_id', 'id']).sort_index()['num_gpus']

,machine_id,id,num_gpus,rentable
0,6312,5516886,4,False
1,6312,5381482,4,False
2,6312,5516889,2,False
3,6312,5516887,2,False
4,6312,5381485,2,False
5,6312,5381483,2,False
6,6312,5516888,1,False
7,6312,5516885,1,False
8,6312,5516884,1,False
9,6312,5516883,1,False


In [27]:
df = np_min_chunk(df)

In [28]:
df_na_vals(df)

is_bid                    ->          Missing values: 0 (0.00%)
inet_up_billed            ->          Missing values: 8 (100.00%)
inet_down_billed          ->          Missing values: 8 (100.00%)
external                  ->          Missing values: 0 (0.00%)
webpage                   ->          Missing values: 8 (100.00%)
logo                      ->          Missing values: 0 (0.00%)
rentable                  ->          Missing values: 0 (0.00%)
compute_cap               ->          Missing values: 0 (0.00%)
credit_balance            ->          Missing values: 8 (100.00%)
credit_discount           ->          Missing values: 8 (100.00%)
credit_discount_max       ->          Missing values: 0 (0.00%)
driver_version            ->          Missing values: 0 (0.00%)
cuda_max_good             ->          Missing values: 0 (0.00%)
machine_id                ->          Missing values: 0 (0.00%)
hosting_type              ->          Missing values: 8 (100.00%)
public_ipaddr             ->

['inet_up_billed',
 'inet_down_billed',
 'webpage',
 'credit_balance',
 'credit_discount',
 'hosting_type',
 'cpu_name',
 'start_date',
 'end_date',
 'duration']

In [29]:
# df[df.machine_id == 10298]
df[df.machine_id == 11734][['start_date', 'end_date']]

,start_date,end_date


In [30]:
!python vast search offers "machine_id = 162 rentable = any verified = any" -d

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%22162%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '162'}, 'order': [['score', 'desc']], 'type': 'on-demand'}
ID       CUDA   N  Model        PCIE  vCPUs   RAM  Disk  $/hr    DLP   DLP/$  NV Driver  Net_up  Net_down  R     Max_Days  mach_id  status    ports  country 
6612968  11.7  4x  RTX_2080_Ti  12.5  32.0   64.4  768   0.6000  49.3  82.08  515.65.01  45.4    345.4     99.8  -         162      verified  1000   Ohio,_US


In [31]:
df.is_bid.any()

False